In [155]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
%matplotlib inline
import scipy.misc
import matplotlib.cm as cm
import random
from scipy.special import expit

## Multi-Class Classification
### Dataset

In [156]:
mat = scipy.io.loadmat('ex3data1.mat')
X, y = mat['X'], mat['y']
# Get a 1's column into X
m = len(y)
ones = np.ones((m,1))
X = np.hstack((ones, X)) #add the intercept
(m,n) = X.shape
print(y.shape) # y is a classification for the image
print(type(y.shape[1]),type(y.shape[0]))
print(X.shape) # X is 5000 images

(5000, 1)
<class 'int'> <class 'int'>
(5000, 401)


In [158]:
# visualizing some part of the data
visual= np.random.choice(X.shape[0],20)
plt.imshow(X[visual,1:].reshape(-5,20).T)
plt.axis('off')

(-0.5, 399.5, 19.5, -0.5)

### Logistic Regression


In [159]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

### Regularized Cost Function
${J(\theta) = (1 \div m) \sum^{m}_{i=1}[-y^{i}log(h_{\theta}(x^i))-(1-y^i)log(1-h_{\theta}(x^i))]+\lambda \div (2m) \sum^n_{j=1}\theta^2_j}$
### Vectorized Cost Function
${J(\theta)=1/m((log(g(X\theta))^Ty+(log(1-g(X\theta))^T(1-y))+\lambda/(2m)\sum^n_{j=1}\theta^2_j}$

In [160]:
def lrcostFuncReg(theta,X,y,lmd):
    #m = len(y)
    temp = sigmoid(X.dot(theta))
    temp1 = np.multiply(y,np.log(temp))
    temp2 = np.multiply(1-y,np.log(1-temp))
    return np.sum(temp1+temp2)/(-m)+np.sum(theta[1:]**2)*lmd/(2*m)

In [161]:
def lrgradientReg(theta,X,y,lmd):
    #m = len(y)
    temp = sigmoid(X.dot(theta))-y
    temp1 = np.dot(temp.T,X).T/m + theta*lmd/m
    temp1[0]=temp1[0] - theta[0]*lmd/m
    return temp1

In [162]:
from scipy.optimize import fmin_cg

#(m,n) = X.shape
lmd=0.1
k=10
theta = np.zeros((k,n))
for i in range(k):
    digit = i if i else 10
    check=(y==digit).flatten()
    theta[i] = fmin_cg(f=lrcostFuncReg,x0=theta[i],fprime = lrgradientReg,args=(X,check,lmd),maxiter=50)

         Current function value: 0.009110
         Iterations: 50
         Function evaluations: 182
         Gradient evaluations: 182
         Current function value: 0.014069
         Iterations: 50
         Function evaluations: 180
         Gradient evaluations: 180
         Current function value: 0.057151
         Iterations: 50
         Function evaluations: 140
         Gradient evaluations: 140
         Current function value: 0.061954
         Iterations: 50
         Function evaluations: 139
         Gradient evaluations: 139
         Current function value: 0.037404
         Iterations: 50
         Function evaluations: 154
         Gradient evaluations: 154
         Current function value: 0.063881
         Iterations: 50
         Function evaluations: 131
         Gradient evaluations: 131
         Current function value: 0.020031
         Iterations: 50
         Function evaluations: 175
         Gradient evaluations: 175
         Current function value: 0.033830
      

In [163]:
pred = np.argmax(X @ theta.T, axis = 1)
pred = [e if e else 10 for e in pred]
np.mean(pred == y.flatten()) * 100

95.08

In [166]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=10,penalty='l2',solver='liblinear')
lr.fit(X[:,1:],y.ravel())
pred_new = lr.predict(X[:,1:])
print('Using Logistic Regression in scikit-learn: {}%'.format(np.mean(pred_new == y.ravel())*100))

C:\Users\LU\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Using Logistic Regression in scikit-learn: 96.5%


## Neural Networks

In [168]:
def predict(theta1,theta2,f):
    z2 = theta1.dot(f.T)
    a2 = np.c_[np.ones((mat['X'].shape[0],1)),sigmoid(z2).T]
    z3 = a2.dot(theta2.T)
    a3 = sigmoid(z3)
    return np.argmax(a3,axis=1)+1
weights = scipy.io.loadmat('ex3weights.mat')
theta1, theta2 = weights['Theta1'],weights['Theta2']
#print(theta1.shape)
#print(theta2.shape)
pred = predict(theta1,theta2,X)
print('Neural Network accuracy:{}%'.format(np.mean(pred==y.ravel())*100))

Neural Network accuracy:97.52%
